In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


Una volta installata la libreria pymongo, si procede con la connessione al nostro database. Attraverso i campi: server, database e collection, ci si collega al database scelto. 

In [2]:
import pymongo
import datetime

# MongoDB Server and Database Data
server = "mongodb://localhost:27017/"
database = "City"
collection = "City_Inspections_DB"

Di seguito le prime funzioni con cui avviene effettivamente la connessione

In [3]:
def connect():
  myclient = pymongo.MongoClient(server)
  mydb = myclient[database]
  mycol = mydb[collection]
  return mycol

Queste sono funzioni di inserimento e aggiornamento dei dati all'interno del database

In [4]:
def insert(mycol):
  inspection = {"id" : "00000-0000-ENFO", "certificate_number" : "17041999", "business_name" : "Videogames Center",
  "date" : "1999-4-17", "result" : "Fail", "sector" : "Videogames", "address" : {"city" : "Georgia",
  "zip" : "41030", "street" : "Rue de Baptiste", "number" : "19"}}  
  mycol.insert_one(inspection)
  

def update(mycol):
  mycol.update_one({"id":"00000-0000-ENFO"},{"$set":{"Inspector Name":"Lorenzo Stigliano"}}, upsert=False)

Queste che seguono sono 3 interrogazioni rappresentative delle potenzialità. All'interno della prima abbiamo una interrogazione riguardo il periodo che comprende 2015-1-1 e 2015-6-30 (year-month-day) e considera tutte le ispezioni che sono risultate come "Fail" o "Violation Issued", fatte a New York a Frederick Douglass Boulevard. Il risultato è stato poi ordinato in base al nome della attività e sono stati stampati solamente i nomi delle attività.

Nella seguente interrogazione vengono invece presi gli oggetti che hanno due determinati zip code: 10030 e 11373 e si selezionano inoltre solamente quelli che hanno come settore quello della vendita ambulante di cibo. Fatto ciò si crea un gruppo con id dell'oggetto e nome dell'attività, si effettua un unwind su questo gruppo e si ordina il risultato in ordine alfabetico crescente.

Per questa interrogazione si prendono invece gli oggetti appartenenti alla città con il seguente zip code: 11234 e si selezionano le attività che lavorano nel settore del "Cigarette Retail Dealer" all'interno della finestra temporale che intercorre tra il 2016-1-1 e il 2016-4-30. Di questi si crea un gruppo basato sull'id dell'oggetto, il nome della via in cui risiede l'attività e il nome stesso dell'attività. Il gruppo viene sottoposto ad una operazione di unwind che permette di effettuare infine un sort sulla somma delle attività con le caratteristiche sopra citate, per ogni via (address.street).

In [8]:
def query(mycol):
    print("First Query:\n")
    result = mycol.find({"date" : {"$gte":"2015-1-1", "$lte":"2015-6-30"}, "result" : {"$in" : ["Fail", "Violation Issued"]}, 
                        "$and" : [{"address.city" : "NEW YORK"}, {"address.street" : "FREDERICK DOUGLASS BLVD"}]}).sort("business_name").distinct("business_name")

    for x in result:
      print(x)  

    print("\n------------------------------------------------------------------\n")
    
    query = [
      {"$match" : {"address.zip" : {"$in" : [10030, 11373]}}},
      {"$match" : {"sector" : "Mobile Food Vendor - 881"}},
      {"$group" : {'_id': '$id', 'inspected_business': {'$addToSet': '$business_name'}}},
      {"$unwind" : "$inspected_business"},
      {'$sort': {'inspected_business': 1}}
    ]

    print("Second Query:\n")

    result = mycol.aggregate(query)

    for x in result:
      print(x)
    
    print("\n------------------------------------------------------------------\n")
    
    query = [
      {"$match" : {"address.zip" : 11234, "sector" : "Cigarette Retail Dealer - 127", "date" : {"$gte":"2016-1-1", "$lte":"2016-4-30"}}},
      {"$group" : { "_id" : "$id", "street_inspected" : {"$addToSet" : '$address.street'}}},
      {"$unwind" : "$street_inspected"},                
      {'$group': {'_id': '$street_inspected', 'count': { '$sum': 1}}},
      {'$sort': {'count': 1}}
    ]

    result = mycol.aggregate(query)

    print("Second Query:\n")

    for x in result:
      print(x)
    
    print("\n------------------------------------------------------------------\n")

Di seguito la funzione main

In [9]:
def main():
  # connect
  print("Trying to connect to: " + str(server))
  mycol = connect()
  print("OK\n")

  # insert
  print("Inserting new object inside: " + str(collection))
  insert(mycol)
  print("OK\n")
  
  # update
  print("Updating the collection: " + str(collection))
  update(mycol)
  print("OK\n")
  
  # query
  print("Queries on collection: " + str(collection))
  query(mycol)

  

if __name__ == "__main__":
  main()

Trying to connect to: mongodb://localhost:27017/
OK

Inserting new object inside: City_Inspections_DB
OK

Updating the collection: City_Inspections_DB
OK

Queries on collection: City_Inspections_DB
First Query:

148 DELI GROCERY, INC.
2117 8TH AVENUE LAUNDROMAT CORP.
2268 DELI & GROCERY INC.
ABDOU KINGS DELI INC.
ABREU, CATALINO
ANAN GOURMET DELI CORP.
APOLLO DELI & FRUIT CORP.
AVON PRODUCTS, INC.
AZAL GROCERY STORE INC
BEST YET MARKET OF HARLEM, INC.
BIER INTERNATIONAL, LLC
BIG APPLE 2 DELI INC.
Best Market
CATALINO ABREU
CJ PARK FOODS INC
CMC NAIL & SPA
COUMBA DIARRA GUEYE
ELHADJ M BARRY
EVERGREEN CLEANERS 144 ST INC.
EXPRESS IVOIR - TOUBALO
FELDER & SONS, INC.
FINE FARE SUPERMARKET
FREDERICK DOUGLAS FURNITURE
L S MOBILE INC.
LD BUSINESS SOLUTIONS
LEGACY CLEANERS INC.
LEGACY WIRELESS INC.
MELO AND FALCON DELI GROCERY CORP.
N&A FOOD MARKET CORP
N.H.A. DELI GROCERY STORE, INC.
NEW BETTY BRITE CLEANERS, INC.
NYC CANDY STORE SHOP CORP
POLO GROUNDS MEAT CORP.
RED LORRY YELLOW LORRY MOVING